In [2]:
import IPython.core.interactiveshell
from agents.tech_data_agent import create_tech_data_agent, tech_data_agent_respond

In [3]:
b = create_tech_data_agent()
b.query("Count the number of Pokémon that can Mega Evolve (have a Mega Evolution form) and also have a Paradox form in Generation 9." )
a = b.get_tool_responses()

[{'id': 'chatcmpl-tool-9618e3043bb8437f8c7d86d123594e9d', 'function': {'arguments': '{"table":"pokemons","columns":["species_name"],"conditions":[{"column":"generation","operator":"=","value":9},{"column":"name","operator":"LIKE","value":"%mega%"}],"condition_logic":"AND","group_by":["species_name"]}', 'name': 'execute_query'}, 'type': 'function', 'index': 0}]
[{'id': 'functions_execute_query_dfad', 'function': {'arguments': '{"table":"pokemons","columns":["species_name"],"conditions":[{"column":"generation","operator":"=","value":9},{"column":"name","operator":"LIKE","value":"%mega%"}],"condition_logic":"AND","group_by":["species_name"]}', 'name': 'execute_query'}, 'type': 'function', 'index': 0}]
[{'id': 'call_ebba7afe00a04807808bbff6', 'function': {'arguments': '{"table": "pokemons", "columns": ["species_name"], "conditions": [{"column": "generation", "operator": "=", "value": 9}, {"column": "name", "operator": "LIKE", "value": "%paradox%"}], "condition_logic": "AND", "group_by": ["

In [7]:
b = tech_data_agent_respond(query="What is the average attack of all fire pokemon with defense < 100?")

[{'id': 'functions_execute_query_a82f', 'function': {'arguments': '{\n  "table": "pokemons",\n  "columns": [\n    {\n      "func": "AVG",\n      "column": "attack"\n    }\n  ],\n  "conditions": [\n    {\n      "column": "type_1",\n      "operator": "=",\n      "value": "fire"\n    },\n    {\n      "column": "defense",\n      "operator": "<",\n      "value": 100\n    },\n    {\n      "column": "is_default",\n      "operator": "=",\n      "value": true\n    }\n  ],\n  "condition_logic": "AND"\n}', 'name': 'execute_query'}, 'type': 'function', 'index': 0}]


In [5]:
a

'Based on the database:\n\n| Pokémon that **can Mega‑Evolve** (include “mega” in the name) | Pokémon that have a **Paradox** form (include “paradox” in the name) |\n|---|---|\n| scovillain‑mega <br>glimmora‑mega <br>tatsugiri‑mega <br>baxcalibur‑mega | None (no Gen\u202f9 Pokémon in the table has “paradox” in its name)\n\nSince there are **no Pokémon in Generation\u202f9** that appear in both categories, the count of Pokémon that can Mega\u202fEvolve **and** also have a Paradox form is:\n\n**0**.'

In [ ]:
a.reasoning_history

['The user asks: "list name, generation, defense where (type_1 = \'fire\' OR type_2 = \'fire\') and defense < 100 order by name". In the schema, we have table pokemons. They want to filter on type_1 = \'fire\' or type_2 = \'fire\', and defense < 100. Order by name.\n\nWe need to use a structured query. They want a mixed OR logic. The tool cannot handle mixed AND/OR. This is a single query: condition is (type_1=\'fire\' OR type_2=\'fire\') AND defense < 100. That is a combination of OR and AND. The tool cannot handle that directly. The guidance says for mixed logic like (A OR B) AND C, we must split into two queries.\n\nSo we choose:\n\nA: type_1=\'fire\'.\nB: type_2=\'fire\'.\nC: defense<100.\n\nWe do Query1: get ids where type_1=\'fire\' AND defense<100.\nQuery2: ids where type_2=\'fire\' AND defense<100.\nThen combine ids, unique.\n\nThen Query3: select name, generation, defense from pokemons where id in combined IDs order by name.\n\nWe need to produce the result using 3 calls.\n\nW